In [1]:
import numpy as np
import scipy.optimize as op
import pandas as pd
import os
import glob
from multiprocessing import Pool

import dynesty
import pickle
import uvlf

In [14]:
processes,nlive=8,8000

DM='WDM'
fs_model='PL'
dataset_name='B'
z_arr=[6,7,8]

#priors:
log10_ks_p1=-2.
log10_ks_p2=1.

a_s1_p1=-1.
a_s1_p2=1.

a_s2_p1=-1.
a_s2_p2=1.


log10_M_с_p1=8.
log10_M_с_p2=10

log10_M_t_p1=8.
log10_M_t_p2=10

mx_p1=0.5
mx_p2=10.

In [7]:
ind=1
if fs_model=='DPLM':
    ind=-1
    

ndim=3
if DM=='WDM':
    ndim+=1
if 'DPL' in fs_model:
    ndim+=2


In [10]:
LF_data=uvlf.dataset(dataset_name,z_arr)

const=0
for z,lf in LF_data.items():
        x=lf[0]
        y=lf[1]
        yerr_down=lf[3]
        yerr_up=lf[2]
        const+=-0.5*len(y)*np.log((np.pi/2))-sum(np.log(yerr_down+yerr_up))


In [11]:
def chi2(y_pred,y, yerr_up, yerr_down):
        
    return np.sum(np.heaviside(y_pred-y,0)*((y_pred-y)/yerr_up)**2+np.heaviside(y-y_pred,0)*((y_pred-y)/yerr_down)**2)

def lnlike(theta):
        
    log10_ks,a_s1,log10_M_t,mx = theta
    M_t=10**log10_M_t
    ks=10**log10_ks
    chi2_sum=0
    for z,lf in LF_data.items():
        x=lf[0]
        y=lf[1]
        yerr_down=lf[3]
        yerr_up=lf[2]
        
        
        
        M_UV_Model, phi_Model = uvlf.compute_LF(z=z, ks=ks,a_s1=a_s1, a_s2=a_s1, M_t=M_t, DM=DM, mx=mx, ind=ind)
    
        y_pred = np.interp( x, M_UV_Model[::-1], np.log10(phi_Model)[::-1] )
#        print(chi2(y_interp,y,yerr_up,yerr_down))
        chi2_sum+=chi2(y_pred,y,yerr_up,yerr_down)
        
    return -0.5*chi2_sum+const

In [12]:
def ptform(u):
    x=np.array(u)
    x[0]=u[0]*(log10_ks_p2-log10_ks_p1)+log10_ks_p1
    x[1]=u[1]*(a_s1_p2-a_s1_p1)+a_s1_p1

    x[2]=u[2]*(log10_M_t_p2-log10_M_t_p1)+log10_M_t_p1
    
    x[3]=u[3]*(mx_p2-mx_p1)+mx_p1
    return x

In [ ]:

with Pool(processes=processes) as pool:
    dsampler = dynesty.DynamicNestedSampler(lnlike, ptform, ndim, pool=pool, queue_size=processes,nlive=nlive)
    dsampler.run_nested(print_progress=True)
dresults = dsampler.results

1132it [17:01,  3.35s/it, batch: 0 | bound: 0 | nc: 4 | ncall: 3979 | eff(%): 25.273 | loglstar:   -inf < -107.012 <    inf | logz: -114.284 +/-  0.158 | dlogz: 123.250 >  0.010]     